## Contents
1. [Algorithm](#algo)
2. [Explanation](#exp)

### Classification Algorithm
<a id='algo'></a>

In [93]:
import cv2
import numpy as np
from skimage import io

image_path = "image.jpg"

def shape_recognition(path, threshold=800):
    '''
    Input:
        path [str]: file path 
        
    Output:
        tuple containing the number of squares, circles and triangle
        
    Method:
        1. Preprocessing of image to eliminate lines/dots in the background and to close up gaps in teh shapes
        2. Drawing contours on potential shapes and approximating these to reduce 'unnatural' edges
        3. For each identified shape, eliminate if the size is too large (detected the frame), or too small (noise)
        4. Count the number of edges for each of these shapes to identify if its a square, circle, or triangle
    '''
    
    # Reading the image in and transforming pixel values into binary for contrast
    
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
    
    # Image prepocessing with morphology transformations
    
    kernel = np.ones((5,5),np.uint8)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

    # Collect the number of edges and size of the shape and fit these into an array
    
    shapes = [] 
    contours,hierarchy = cv2.findContours(img, 1, 2)
    # for each contour,approximate the outlines and count the edges and surface area
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
        cv2.drawContours(img, [approx], 0, (1), 5)
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        shapes.append((cv2.contourArea(cnt), len(approx)))
        
    # Find shapes with area above a threshold value, and classify based on no. of edges
    
    squares, circles, triangles = 0,0,0
    shapes.sort(key=lambda x: x[0])
    for shape in shapes[::-1]:
        if shape[0] < 240000 and shape[0] > threshold:
            if shape[1] == 3:
                triangles += 1
            elif shape[1] == 4:
                squares += 1
            else:
                circles += 1
                
    return (squares, circles, triangles)
        

shape_recognition(image_path)

(0, 2, 0)

### Further Explanation
<a id='exp'></a>

#### Choice of Algorithm:

I chose the cv2 library to draw a contour over potential shapes and this is done through a topological analysis of digitized binary images, thus making it one of the more efficient solutions. To maximize accuracy, the images should be as clean as possible, so that background noise will not be identified as shapes. This is done through morphological transformation that processes binary pixels against their neighbours to 'close' gaps and 'open' up shape links. However, the denoising is not perfect; many of the images still retain severed lines/dots after the processing. Hence, a threshold is defined to eliminate shapes that are too small to avoid non-shapes. The downside to this is that smaller shapes will be ignored. The threshold thus serve as a control for false positive vs false negatives. If cleaning was more properly done, this will not be needed. Alternatively, instead of making use of size, we can check whether the shape detected is solid, in other words, if its center pixels have the same color as its perimeter. I have not yet found a good way of dealing with this while using cv2.findContours. Another approach I have considered is to use a gaussian blur to remove the noise, but this will compromise the struture of the shapes, leading to more than expected number of edges. 

#### Alternative Considerations:

The first thought that came to mind for shape detection is to use image gradients to detect the directions of each shape. Feature detection algorithms like SIFT might be considered but I am not sure of its performance on binary images. For a brief moment, I did consider using neural networks but that seemed slightly excessive since that will require a lot of data and images with pure shapes will need manual generation and that seems like a lot of work. Also, since shape detection ultimately have some underlying maths explanation (circles have ~10 edges approximated, while squares have 4, or squares have 90 degress slants while triangles have <90 degree slopes), it might be better to uncover these using algorithms than NNs. 
